In [1]:
import re

filepath = "/workspace/DeepBench/code/nvidia/dbnvidiaresnext.txt"

data = []
with open(filepath, 'r') as file:
    # this file is usually small => can read all at once
    lines = file.readlines()
#     headers = lines[3]
    lines = lines[5:]
    # w,h,c,n,k,f_w,f_h,pad_w,pad_h,stride_w,stride_h,group,precision,fwd_time(usec),bw_inputs_time(usec),bwd_params_time(usec),total_time(usec),fwd_algo,bwd_inputs_algo,bwd_params_algo
    pattern = re.compile(r'\s*(\d+)\s+(\d+)\s+(\d+)\s+(\d+)\s+(\d+)\s+(\d+)\s+(\d+)\s+(\d+)\s+(\d+)\s+(\d+)\s+(\d+)\s+(\d+)\s+(\w+)\s+(\d+)\s+(\d+)\s+(\d+)\s+(\d+)\s+(\w+)\s+(\w+)\s+(\w+)\s*')
    for line in lines:
        info = pattern.match(line)
#         print(info.groups())
        data.append(info.groups())

In [2]:
import pandas as pd

times = pd.DataFrame(data, columns=["w","h","c","n","k","f_w","f_h","pad_w","pad_h","stride_w","stride_h","group","precision", \
                                    "fwd_time (usec)","bwd_inputs_time (usec)","bwd_params_time (usec)","total_time (usec)", \
                                    "fwd_algo","bwd_inputs_algo","bwd_params_algo"])

numeric_cols = [col for col in times.columns.tolist() if col not in ['precision', 'fwd_algo', 'bwd_inputs_algo', 'bwd_params_algo']]
times[numeric_cols] = times[numeric_cols].apply(pd.to_numeric)
print(times)

      w    h      c  n      k  f_w  f_h  pad_w  pad_h  stride_w  stride_h  \
0   320  320      3  4     64    7    7      3      3         2         2   
1    80   80     64  4   1536    1    1      0      0         1         1   
2    80   80   1536  4   1536    3    3      1      1         1         1   
3    80   80   1536  4    256    1    1      0      0         1         1   
4    80   80     64  4    256    1    1      0      0         1         1   
5    80   80    256  4   1536    1    1      0      0         1         1   
6    80   80    256  4   3072    1    1      0      0         1         1   
7    80   80   3072  4   3072    3    3      1      1         2         2   
8    40   40   3072  4    512    1    1      0      0         1         1   
9    80   80    256  4    512    1    1      0      0         2         2   
10   40   40    512  4   3072    1    1      0      0         1         1   
11   40   40   3072  4   3072    3    3      1      1         1         1   

In [3]:
archinfo = pd.read_pickle('/workspace/resnext101_32x48d-archinfo.pkl')
print(archinfo)

     n      c    h    w      k  f_h  f_w  pad_h  pad_w  stride_h  stride_w  \
0    4      3  320  320     64    7    7      3      3         2         2   
1    4     64   80   80   1536    1    1      0      0         1         1   
2    4   1536   80   80   1536    3    3      1      1         1         1   
3    4   1536   80   80    256    1    1      0      0         1         1   
4    4     64   80   80    256    1    1      0      0         1         1   
..  ..    ...  ...  ...    ...  ...  ...    ...    ...       ...       ...   
99   4  12288   10   10  12288    3    3      1      1         1         1   
100  4  12288   10   10   2048    1    1      0      0         1         1   
101  4   2048   10   10  12288    1    1      0      0         1         1   
102  4  12288   10   10  12288    3    3      1      1         1         1   
103  4  12288   10   10   2048    1    1      0      0         1         1   

     dil_h  dil_w  group                                       

In [5]:
# took me > 1 hour to write this line of code. fml
diffconvs = archinfo.groupby(archinfo.columns.tolist(),as_index=False).size().reset_index().rename(columns={0:'repetitions'})
if 'dil_h' in archinfo.columns:
    diffconvs.drop(['dil_h'], axis=1, inplace=True)
if 'dil_w' in archinfo.columns:
    diffconvs.drop(['dil_w'], axis=1, inplace=True)
if 'fwd_algo' in archinfo.columns:
    diffconvs.drop(['fwd_algo'], axis=1, inplace=True)
print(diffconvs)

    n      c    h    w      k  f_h  f_w  pad_h  pad_w  stride_h  stride_w  \
0   4      3  320  320     64    7    7      3      3         2         2   
1   4     64   80   80    256    1    1      0      0         1         1   
2   4     64   80   80   1536    1    1      0      0         1         1   
3   4    256   80   80    512    1    1      0      0         2         2   
4   4    256   80   80   1536    1    1      0      0         1         1   
5   4    256   80   80   3072    1    1      0      0         1         1   
6   4    512   40   40   1024    1    1      0      0         2         2   
7   4    512   40   40   3072    1    1      0      0         1         1   
8   4    512   40   40   6144    1    1      0      0         1         1   
9   4   1024   20   20   2048    1    1      0      0         2         2   
10  4   1024   20   20   6144    1    1      0      0         1         1   
11  4   1024   20   20  12288    1    1      0      0         1         1   

In [10]:
joincols = [col for col in diffconvs.columns.tolist() if col != "repetitions"]
model_total = pd.merge(times, diffconvs, on=joincols, how='inner')
model_total["fwd_time_with_repetitions(msec)"] = model_total["fwd_time (usec)"]
model_total["total_time_with_repetitions(msec)"] = model_total["total_time (usec)"] * model_total["repetitions"]
display(model_total)


,w,h,c,n,k,f_w,f_h,pad_w,pad_h,stride_w,...,precision,fwd_time (usec),bwd_inputs_time (usec),bwd_params_time (usec),total_time (usec),fwd_algo,bwd_inputs_algo,bwd_params_algo,repetitions,Total Time with repetitions (msec)
0,320,320,3,4,64,7,7,3,3,2,...,float,229,550,409,1188,IMPLICIT_PRECOMP_GEMM,AL0,AL0,1,1.188
1,80,80,64,4,1536,1,1,0,0,1,...,float,548,574,749,1871,IMPLICIT_PRECOMP_GEMM,AL1,AL0,1,1.871
2,80,80,1536,4,1536,3,3,1,1,1,...,float,2800,2796,6780,12376,WINOGRAD,WINOGRAD,AL0,3,37.128
3,80,80,1536,4,256,1,1,0,0,1,...,float,2211,1962,2612,6785,IMPLICIT_PRECOMP_GEMM,AL1,AL0,3,20.355
4,80,80,64,4,256,1,1,0,0,1,...,float,109,117,233,459,IMPLICIT_PRECOMP_GEMM,AL1,AL0,1,0.459
5,80,80,256,4,1536,1,1,0,0,1,...,float,1941,2260,2593,6794,IMPLICIT_PRECOMP_GEMM,AL1,AL0,2,13.588
6,80,80,256,4,3072,1,1,0,0,1,...,float,3833,4468,5061,13362,IMPLICIT_PRECOMP_GEMM,AL1,AL0,1,13.362
7,80,80,3072,4,3072,3,3,1,1,2,...,float,3783,9022,5516,18321,IMPLICIT_PRECOMP_GEMM,AL0,AL0,1,18.321
8,40,40,3072,4,512,1,1,0,0,1,...,float,2514,1954,2577,7045,IMPLICIT_PRECOMP_GEMM,AL1,AL0,4,28.180
9,80,80,256,4,512,1,1,0,0,2,...,float,187,332,329,848,IMPLICIT_PRECOMP_GEMM,AL1,AL0,1,0.848


In [11]:
large_time=model_total[model_total['Total Time with repetitions (msec)']>100]
display(large_time)

,w,h,c,n,k,f_w,f_h,pad_w,pad_h,stride_w,...,precision,fwd_time (usec),bwd_inputs_time (usec),bwd_params_time (usec),total_time (usec),fwd_algo,bwd_inputs_algo,bwd_params_algo,repetitions,Total Time with repetitions (msec)
14,20,20,6144,4,1024,1,1,0,0,1,...,float,2233,2024,2582,6839,IMPLICIT_PRECOMP_GEMM,AL1,AL0,23,157.297
16,20,20,1024,4,6144,1,1,0,0,1,...,float,2021,2234,2565,6820,IMPLICIT_PRECOMP_GEMM,AL1,AL0,22,150.040
17,20,20,6144,4,6144,3,3,1,1,1,...,float,3633,3632,2370,9635,IMPLICIT_PRECOMP_GEMM,AL1,WINOGRAD_NONFUSED,22,211.970
